# KPI de Profit Total (CLP)

Este cuaderno calcula el indicador financiero solicitado utilizando los datos de `codigos pasados /merged_customers_with_effective.csv`. El KPI considera únicamente a los clientes que completaron su servicio (`outcome == "served"`).

In [1]:
from pathlib import Path
import pandas as pd

data_path = Path('codigos pasados ') / 'merged_customers_with_effective.csv'
df = pd.read_csv(data_path)
df.head()

,source_folder,customer_id,profile,priority,items,payment_method,arrival_time_s,service_start_s,service_end_s,wait_time_s,...,total_profit_clp,cart_category_profit_clp,lane_name,lane_type,queue_request_priority,patience_s,outcome,balk_reason,abandon_reason,effective_queue_length
0,Week-1-Day-1,1,regular,no_priority,12,card,55.184350,55.184350,149.959594,0.0,...,2870.0,"{""produce"": 190, ""grocery"": 640, ""meat"": 0, ""d...",regular-1,regular,1.0,412.595926,served,NaN,NaN,0.0
1,Week-1-Day-1,2,regular,no_priority,25,card,83.543164,83.543164,307.117844,0.0,...,10890.0,"{""produce"": 200, ""grocery"": 330, ""meat"": 2040,...",regular-2,regular,1.0,428.983308,served,NaN,NaN,0.0
2,Week-1-Day-1,3,express_basket,no_priority,13,card,97.778322,97.778322,239.331803,0.0,...,3410.0,"{""produce"": 70, ""grocery"": 0, ""meat"": 0, ""dair...",regular-10,regular,1.0,270.415223,served,NaN,NaN,0.0
3,Week-1-Day-1,4,family_cart,no_priority,78,cash,145.223162,145.223162,406.030979,0.0,...,18410.0,"{""produce"": 1110, ""grocery"": 1290, ""meat"": 289...",regular-5,regular,1.0,748.890609,served,NaN,NaN,0.0
4,Week-1-Day-1,5,express_basket,no_priority,1,card,216.473592,216.473592,244.596471,0.0,...,380.0,"{""produce"": 0, ""grocery"": 0, ""meat"": 0, ""dairy...",express-1,express,1.0,24.638833,served,NaN,NaN,0.0


El KPI se define como:

$$PROF_{CLP} = \sum_{i \in S} total\_profit\_clp_i$$

donde $S$ representa el conjunto de clientes servidos.

In [2]:
served_df = df[df['outcome'] == 'served']
prof_clp = served_df['total_profit_clp'].sum()
prof_clp

8914190130.0

In [3]:
print(f'PROF_CLP (clientes servidos): {prof_clp:,.0f} CLP')

PROF_CLP (clientes servidos): 8,914,190,130 CLP


## Profit mensual (agrupado por 4 semanas)

Para analizar la tendencia mensual se agrupan las semanas de la simulación en bloques de 4 semanas y se calcula el `PROF_CLP` para los clientes servidos en cada bloque.

In [4]:
import re

served_df = df[df['outcome'] == 'served'].copy()
served_df['week'] = served_df['source_folder'].str.extract(r'Week-(\d+)')
served_df['week'] = served_df['week'].astype('Int64')
served_df['month_4w'] = ((served_df['week'] - 1) // 4 + 1).astype('Int64')
monthly_prof = (
    served_df.groupby('month_4w', dropna=True)['total_profit_clp']
    .sum()
    .reset_index(name='prof_clp')
    .sort_values('month_4w')
)
monthly_prof

,month_4w,prof_clp
0,1,684772200.0
1,2,688379780.0
2,3,690168780.0
3,4,685124970.0
4,5,688824850.0
5,6,685141770.0
6,7,686696220.0
7,8,683861480.0
8,9,685321100.0
9,10,680350660.0


In [5]:
monthly_prof.assign(
    prof_clp_fmt=monthly_prof['prof_clp'].map(lambda x: f"{x:,.0f} CLP")
)

,month_4w,prof_clp,prof_clp_fmt
0,1,684772200.0,"684,772,200 CLP"
1,2,688379780.0,"688,379,780 CLP"
2,3,690168780.0,"690,168,780 CLP"
3,4,685124970.0,"685,124,970 CLP"
4,5,688824850.0,"688,824,850 CLP"
5,6,685141770.0,"685,141,770 CLP"
6,7,686696220.0,"686,696,220 CLP"
7,8,683861480.0,"683,861,480 CLP"
8,9,685321100.0,"685,321,100 CLP"
9,10,680350660.0,"680,350,660 CLP"


> La columna `month_4w` indica el bloque de 4 semanas (mes operativo) y `prof_clp` muestra el profit total en pesos chilenos para los clientes servidos en ese periodo.

## Tasa de Abandono por Perfil (TAC_p)

La tasa \(TAC_p\) mide el porcentaje de clientes de un perfil `p` que no concretan la compra. Se consideran los tres posibles resultados almacenados en la columna `outcome` (`served`, `abandoned`, `balked`):

$$TAC_p = \frac{N_{abandoned,p} + N_{balked,p}}{N_{served,p} + N_{abandoned,p} + N_{balked,p}} \times 100\%$$

Esta métrica ayuda a identificar qué perfiles pierden más ventas por abandono o por no ingresar a la fila.

In [11]:
status_cols = ['served', 'abandoned', 'balked']
profile_counts = (
    df[df['outcome'].isin(status_cols)]
    .groupby('profile')['outcome']
    .value_counts()
    .unstack(fill_value=0)
)
# Aseguramos que todas las columnas existan
for col in status_cols:
    if col not in profile_counts.columns:
        profile_counts[col] = 0
profile_counts = profile_counts[status_cols]

tac_profile = profile_counts.assign(
    total=lambda d: d.sum(axis=1),
    tac=lambda d: ((d['abandoned'] + d['balked']) / d['total'] * 100).round(2)
)

tac_summary = tac_profile[['tac']].assign(
    tac_pct=lambda d: d['tac'].map(lambda x: f"{x:.2f}%")
)[['tac_pct']]

profile_counts, tac_summary

(outcome            served  abandoned  balked
 profile                                     
 deal_hunter        216726      37822    1988
 express_basket     382809      19967    1406
 family_cart        166729      46505       0
 regular            183287      45637      14
 self_checkout_fan  125542       6420      46
 weekly_planner     166972      54793       0,
 outcome           tac_pct
 profile                  
 deal_hunter        15.52%
 express_basket      5.29%
 family_cart        21.81%
 regular            19.94%
 self_checkout_fan   4.90%
 weekly_planner     24.71%)

In [12]:
tac_profile_display = (
    tac_profile.assign(
        served_pct=(tac_profile['served'] / tac_profile['total'] * 100).round(2),
        abandoned_pct=(tac_profile['abandoned'] / tac_profile['total'] * 100).round(2),
        balked_pct=(tac_profile['balked'] / tac_profile['total'] * 100).round(2)
    )
    .rename(columns={
        'tac': 'tac_pct',
        'served': 'served_n',
        'abandoned': 'abandoned_n',
        'balked': 'balked_n'
    })
)

tac_profile_display.assign(
    tac_pct=lambda d: d['tac_pct'].map(lambda x: f"{x:.2f}%"),
    served_pct=lambda d: d['served_pct'].map(lambda x: f"{x:.2f}%"),
    abandoned_pct=lambda d: d['abandoned_pct'].map(lambda x: f"{x:.2f}%"),
    balked_pct=lambda d: d['balked_pct'].map(lambda x: f"{x:.2f}%")
)

outcome,served_n,abandoned_n,balked_n,total,tac_pct,served_pct,abandoned_pct,balked_pct
profile,,,,,,,,
deal_hunter,216726,37822,1988,256536,15.52%,84.48%,14.74%,0.77%
express_basket,382809,19967,1406,404182,5.29%,94.71%,4.94%,0.35%
family_cart,166729,46505,0,213234,21.81%,78.19%,21.81%,0.00%
regular,183287,45637,14,228938,19.94%,80.06%,19.93%,0.01%
self_checkout_fan,125542,6420,46,132008,4.90%,95.10%,4.86%,0.03%
weekly_planner,166972,54793,0,221765,24.71%,75.29%,24.71%,0.00%


> La primera tabla muestra el número de clientes por resultado para cada perfil y la segunda resume `TAC_p` como porcentaje de abandono junto con la distribución de outcomes.

## Tiempo Medio de Espera por Perfil (TMC_p)

El indicador \(TMC_p\) calcula el tiempo promedio de espera en cola para los clientes del perfil `p` que fueron atendidos (`outcome == 'served'`). Se obtiene directamente de la columna `wait_time_s`:

$$TMC_p = \frac{1}{|S_p|} \sum_{i \in S_p} wait\_time\_s_i$$

Donde \(S_p\) es el conjunto de clientes servidos del perfil `p`.

In [13]:
tmc_df = (
    df[df['outcome'] == 'served']
    .groupby('profile')['wait_time_s']
    .mean()
    .reset_index(name='tmc_wait_time_s')
)

# Tabla con formato amigable (minutos y segundos)
def fmt_seconds(seconds):
    minutes = int(seconds // 60)
    secs = seconds % 60
    return f"{minutes}m {secs:05.2f}s"

tmc_df.assign(tmc_fmt=tmc_df['tmc_wait_time_s'].map(fmt_seconds)).sort_values('tmc_wait_time_s')


,profile,tmc_wait_time_s,tmc_fmt
1,express_basket,6.640934,0m 06.64s
4,self_checkout_fan,16.271997,0m 16.27s
0,deal_hunter,29.491098,0m 29.49s
3,regular,89.581731,1m 29.58s
5,weekly_planner,134.828897,2m 14.83s
2,family_cart,144.671696,2m 24.67s


> `tmc_wait_time_s` está expresado en segundos; la columna `tmc_fmt` muestra el mismo promedio en formato minutos:segundos para facilitar la lectura.

## Tasa de Abandono de Clientes con Alto Volumen de Ítems (TAC_HV)

Este indicador mide el porcentaje de clientes con carritos grandes (`items >= 40`) que no completan su compra. Se consideran los resultados `abandoned` y `balked` frente al total de clientes con alto volumen, donde el total incluye también a quienes fueron atendidos (`served`).

$$TAC_{HV} = \frac{N_{abandoned}^{(\geq 40)} + N_{balked}^{(\geq 40)}}{N_{served}^{(\geq 40)} + N_{abandoned}^{(\geq 40)} + N_{balked}^{(\geq 40)}} \times 100\%$$

El análisis se presenta por tipo de perfil (`profile`) para identificar qué segmentos pierden más compras de alto valor.

In [14]:
hv_outcomes = ['served', 'abandoned', 'balked']
hv_df = df[(df['items'] >= 40) & (df['outcome'].isin(hv_outcomes))]

hv_counts = (
    hv_df.groupby('profile')['outcome']
    .value_counts()
    .unstack(fill_value=0)
)
for col in hv_outcomes:
    if col not in hv_counts.columns:
        hv_counts[col] = 0
hv_counts = hv_counts[hv_outcomes]

hv_stats = hv_counts.assign(
    total=lambda d: d.sum(axis=1),
    tac_hv=lambda d: ((d['abandoned'] + d['balked']) / d['total'] * 100).round(2)
)

hv_summary = hv_stats[['tac_hv']].assign(
    tac_hv_pct=lambda d: d['tac_hv'].map(lambda x: f"{x:.2f}%")
)[['tac_hv_pct']]

hv_counts, hv_summary

(outcome            served  abandoned  balked
 profile                                     
 deal_hunter          4541       2177     105
 express_basket         57         28       2
 family_cart        151025      42253       0
 regular             41145      13092       4
 self_checkout_fan      75         33       2
 weekly_planner     124043      41606       0,
 outcome           tac_hv_pct
 profile                     
 deal_hunter           33.45%
 express_basket        34.48%
 family_cart           21.86%
 regular               24.14%
 self_checkout_fan     31.82%
 weekly_planner        25.12%)

In [15]:
hv_stats_display = (
    hv_stats.rename(columns={
        'served': 'served_n',
        'abandoned': 'abandoned_n',
        'balked': 'balked_n',
        'tac_hv': 'tac_hv_pct'
    })
)

hv_stats_display.assign(
    tac_hv_pct=lambda d: d['tac_hv_pct'].map(lambda x: f"{x:.2f}%"),
    served_pct=lambda d: (d['served_n'] / d['total'] * 100).map(lambda x: f"{x:.2f}%"),
    abandoned_pct=lambda d: (d['abandoned_n'] / d['total'] * 100).map(lambda x: f"{x:.2f}%"),
    balked_pct=lambda d: (d['balked_n'] / d['total'] * 100).map(lambda x: f"{x:.2f}%")
)

outcome,served_n,abandoned_n,balked_n,total,tac_hv_pct,served_pct,abandoned_pct,balked_pct
profile,,,,,,,,
deal_hunter,4541,2177,105,6823,33.45%,66.55%,31.91%,1.54%
express_basket,57,28,2,87,34.48%,65.52%,32.18%,2.30%
family_cart,151025,42253,0,193278,21.86%,78.14%,21.86%,0.00%
regular,41145,13092,4,54241,24.14%,75.86%,24.14%,0.01%
self_checkout_fan,75,33,2,110,31.82%,68.18%,30.00%,1.82%
weekly_planner,124043,41606,0,165649,25.12%,74.88%,25.12%,0.00%


> La primera tabla resume los conteos de resultados para clientes con `items >= 40` por perfil; la segunda incorpora el `TAC_HV` en porcentaje junto con la distribución porcentual de cada outcome.